<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks

- Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
- For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 9 Assignment: Detect Multiple Faces**

**Student Name: Your Name**


# Assignment Instructions

I provide you with five images of people in various poses, such as the following:

![Multiple Faces](https://data.heatonresearch.com/images/wustl/data/AdobeStock_319245189-low.jpg)

As you have seen from module 9, you can detect faces in this image, as you can see here:

![Multiple Faces](https://s3.amazonaws.com/data.heatonresearch.com/images/wustl/class/bound-1.jpg)

Your task for this assignment is to extract the coordinates (x,y) and dimensions (height, width). Extract the dimensions/coordinates for a rectangle around the individual face rectangles. This rectangle will overlap the edges of some of the separate face rectangles. The single rectangle would look similar, though you do not need to draw it. Rather, you will return a data frame of the coordinates and dimensions.

![Multiple Faces](https://s3.amazonaws.com/data.heatonresearch.com/images/wustl/class/bound-2.jpg)

Generate your dataframe from the following images.

- https://data.heatonresearch.com/images/wustl/data/AdobeStock_158302589-low.jpg
- https://data.heatonresearch.com/images/wustl/data/AdobeStock_268797955-low.jpg
- https://data.heatonresearch.com/images/wustl/data/AdobeStock_319245189-low.jpg
- https://data.heatonresearch.com/images/wustl/data/AdobeStock_622573012-low.jpg
- https://data.heatonresearch.com/images/wustl/data/AdobeStock_632061559-low.jpg

Your submitted dataframe should look like this. Make sure to round your numbers and convert to integer. You will need to calculate the width and height.

| x   | y   | width | height |
| --- | --- | ----- | ------ |
| 177 | 215 | 614   | 134    |
| 316 | 74  | 472   | 231    |
| 231 | 59  | 497   | 264    |
| 436 | 160 | 167   | 245    |
| 140 | 192 | 760   | 252    |


# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to `/content/drive`.


In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch

device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: cuda


# Assignment Submit Function

You will submit the 10 programming assignments electronically. The following submit function can be used to do this. My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**


In [2]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Install Facenet-Pytorch


In [3]:
!pip install facenet-pytorch

# Assignment #9 Sample Code

The following code provides a starting point for this assignment.


In [23]:
import os
import pandas as pd
import torch
from facenet_pytorch import MTCNN
from PIL import Image
import requests
from io import BytesIO
import torch
from PIL import Image, ImageDraw

# Load image
urls = ['https://data.heatonresearch.com/images/wustl/data/AdobeStock_158302589-low.jpg',
       'https://data.heatonresearch.com/images/wustl/data/AdobeStock_268797955-low.jpg',
       'https://data.heatonresearch.com/images/wustl/data/AdobeStock_319245189-low.jpg',
       'https://data.heatonresearch.com/images/wustl/data/AdobeStock_622573012-low.jpg',
       'https://data.heatonresearch.com/images/wustl/data/AdobeStock_632061559-low.jpg']
df = pd.DataFrame(columns = ['x', 'y', 'width', 'height'])
for url, i in zip(urls, range(5)):
    response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    img = Image.open(BytesIO(response.content))
    img.load()

    # Detect faces
    mtcnn = MTCNN(keep_all=True, device=device)
    boxes, _ = mtcnn.detect(img)
    temp = np.array(boxes)
    min_values_first_two_columns = np.min(temp[:, :2], axis=0)
    max_values_last_two_columns = np.max(temp[:, -2:], axis=0)
    width_height = max_values_last_two_columns - min_values_first_two_columns
    new_df = pd.DataFrame({'x': min_values_first_two_columns[0],
                       'y': min_values_first_two_columns[1],
                       'width': width_height[0],
                       'height': width_height[1]}, index=[i])
    df = pd.concat([df, new_df], ignore_index=True)
print(df)



            x           y       width      height
0  176.703796  214.599670  614.213867  133.588776
1  315.859985   73.516212  472.471558  231.192589
2  231.127029   59.369225  496.964523  264.050148
3  436.048737  160.210815  167.094208  244.645325
4  140.156952  191.794403  759.977692  252.469055


In [22]:
import os
import pandas as pd
import torch
from facenet_pytorch import MTCNN
from PIL import Image
import requests
from io import BytesIO
import torch
from PIL import Image, ImageDraw
from facenet_pytorch import MTCNN

# This is your student key that I emailed to you at the beginnning of the semester.
key = "r2nrqz2pX53SGKnwA07UW52mBbNzuLpf8e2ZYIV9"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='C:\\Users\\bloss\\OneDrive\\桌面\\T81-558\\assignment_Zixuan_class9.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class9.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class9.ipynb'  # Mac/Linux

df_submit = df.round()

# Submit
submit(source_file=file,data=[df_submit],key=key,no=9)

Success: Submitted Assignment 9 for n.zixuan:
You have submitted this assignment 3 times. (this is fine)
No errors, warnings, or notes on your data. Rock on! You will probably do well, but no guarantee. :-)
